## PseudoBulk RNAseq Test:
#### Goal:
Take single cell RNA-sequencing (scRNAseq) data, convert to pseudobulk data by a few different methods (geometric mean (drop 0’s), arithmetic mean, and harmonic means) for each person, then see how/where these individual patients cluster in the overall disease RNA-seq dataset.  
#### Dataset(s):
##### Triple Negative Breast Cancer (TNBC)
Single cell RNA sequencing of 1,534 cells in six fresh triple negative breast cancer tumors.  Download the GSE11839_tpm_rsem.txt data file. 

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE118389  

Karaayvaz M, Cristea S, Gillespie SM, Patel AP et al. Unravelling subclonal heterogeneity and aggressive disease states in TNBC through single-cell RNA-seq. Nat Commun 2018 Sep 4;9(1):3588. PMID: 30181541

<b>Step 1:</b> Load necessary libraries to run analysis

In [27]:
using DelimitedFiles, Statistics, StatsBase;

<b>Step 2:</b> Load raw data of single cell RNA-seq counts. <i><u> Note:</u> this tool cannot handle raw scRNAseq data in a FASTQ format, but often only the RNA counts are freely available in online databases, because the raw data is a protected format.</i>

In [2]:
path = "/github/pseudoBulk/"           # update the path here
file = "GSE118389_tpm_rsem.txt"
infile = string(path,file)
delim = '\t'
rawWithHeader = readdlm(infile, delim);

Pull out the patient labels and the gene names from the RNAseq counts data

In [3]:
sampleNames = rawWithHeader[1,:];
println(sampleNames[1:10])
geneNames = rawWithHeader[2:end,1];
println(geneNames[1:10])

Any["PT089_P1_A01", "PT089_P1_A02", "PT089_P1_A03", "PT089_P1_A04", "PT089_P1_A05", "PT089_P1_A06", "PT089_P1_A07", "PT089_P1_A08", "PT089_P1_A09", "PT089_P1_A10"]
Any["A1BG", "A1BG-AS1", "A1CF", "A2M", "A2M-AS1", "A2ML1", "A2MP1", "A3GALT2", "A4GALT", "A4GNT"]


In [64]:
raw = rawWithHeader[2:end,2:end];
raw = convert(Array{Float64},raw);
println(raw[1:10,1:10])

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 224.88; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.41 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 281.39 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.25 0.0 0.0 0.0 0.0 0.11 0.31 0.53 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


Check the size of the data, as reported in GEO for this dataset, there was single cell RNA-seq data for 1534 distict cells.

In [65]:
size(raw)

(21785, 1534)

<b>Step 3:</b> Identify distict patient data and subset full dataset accordingly.

<i>From the sampleName list, determine the unique patient identifier (in this case the first string before the underscore)</i>

In [6]:
name = Array{Any}(undef, length(sampleNames)-1, 1)
for i=1:(length(sampleNames)-1)
    name[i] = split(sampleNames[i],"_")[1]
end

In [7]:
# identifying unique patient identifiers
uniquePtList = unique(name)

6-element Array{Any,1}:
 "PT089"
 "PT039"
 "PT058"
 "PT081"
 "PT084"
 "PT126"

<i> Once the patients have been identified, create an empty array for each patient. Then fill arrays with each column of scRNAseq count data corresponding to that patient identifier. Repeat for each patient in the dataset. </i>

In [66]:
# collecting individual patient single cell RNAseq counts
global TNBCpt1 = Array{Float64}(undef,size(raw)[1],0)
global TNBCpt2 = Array{Float64}(undef,size(raw)[1],0)
global TNBCpt3 = Array{Float64}(undef,size(raw)[1],0)
global TNBCpt4 = Array{Float64}(undef,size(raw)[1],0)
global TNBCpt5 = Array{Float64}(undef,size(raw)[1],0)
global TNBCpt6 = Array{Float64}(undef,size(raw)[1],0)

21785×0 Array{Float64,2}

In [67]:
for j=1:(length(sampleNames)-1)                # for each column in the overall raw data matrix
    if name[j] == uniquePtList[1]              # if the unique identifier for that column matches patient 1
        global TNBCpt1 = [TNBCpt1 raw[:,j]]    #     then add that column to the patient's scRNAseq array
        end                                    #     if not, skip over that column 
end
size(TNBCpt1)                                  # printout final dimensions for patient 1 (geneNum, cellNum)

(21785, 333)

In [68]:
for j=1:(length(sampleNames)-1)
    if name[j] == uniquePtList[2]
        global TNBCpt2 = [TNBCpt2 raw[:,j]]
    end
end
size(TNBCpt2)

(21785, 341)

In [69]:
for j=1:(length(sampleNames)-1)
    if name[j] == uniquePtList[3]
        global TNBCpt3 = [TNBCpt3 raw[:,j]]
    end
end
size(TNBCpt3)

(21785, 96)

In [70]:
for j=1:(length(sampleNames)-1)
    if name[j] == uniquePtList[4]
        global TNBCpt4 = [TNBCpt4 raw[:,j]]
    end
end
size(TNBCpt4)

(21785, 288)

In [71]:
for j=1:(length(sampleNames)-1)
    if name[j] == uniquePtList[5]
        global TNBCpt5 = [TNBCpt5 raw[:,j]]
    end
end
size(TNBCpt5)

(21785, 286)

In [72]:
for j=1:(length(sampleNames)-1)
    if name[j] == uniquePtList[6]
        global TNBCpt6 = [TNBCpt6 raw[:,j]]
    end
end
size(TNBCpt6)

(21785, 190)

<b>Step 4:</b> Pseudo-bulk data by averaging together the counts across cells for each gene.

<i> First create a function to save the data after each type of mean was calculated. </i>

In [155]:
function saveData(data,ptNum,typeMean)
    dataOut = Array{Any}(undef,size(raw)[1],2);
    dataOut[:,1] = geneNames;
    dataOut[:,2] = data[:,1];
    header = ["" uniquePtList[ptNum]]
    dataOut = [header; dataOut]
    outfile = string(path,uniquePtList[ptNum],"_bulk",typeMean,".txt")
    writedlm(outfile, dataOut, delim)
end

saveData (generic function with 1 method)

<i><u> Arithmetic Mean </u></i>

In [105]:
## initialize the patient specific pseudo-bulk RNA
global TNBCpt1AM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt2AM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt3AM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt4AM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt5AM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt6AM = Array{Float64}(undef,size(raw)[1],1);

In [169]:
for k=1:(length(geneNames))                       # for each row in the overall patient's data matrix
    global TNBCpt1AM[k,1] = mean(TNBCpt1[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt1AM,1,"ArithmeticMean")            # save the data

In [170]:
for k=1:(length(geneNames))                       # for each row in the overall patient's data matrix
    global TNBCpt2AM[k,1] = mean(TNBCpt2[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt2AM,2,"ArithmeticMean")            # save the data

In [171]:
for k=1:(length(geneNames))                       # for each row in the overall patient's data matrix
    global TNBCpt3AM[k,1] = mean(TNBCpt3[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt3AM,3,"ArithmeticMean")            # save the data

In [172]:
for k=1:(length(geneNames))                       # for each row in the overall patient's data matrix
    global TNBCpt4AM[k,1] = mean(TNBCpt4[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt4AM,4,"ArithmeticMean")            # save the data 

In [173]:
for k=1:(length(geneNames))                       # for each row in the overall patient's data matrix
    global TNBCpt5AM[k,1] = mean(TNBCpt5[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt5AM,5,"ArithmeticMean")            # save the data

In [174]:
for k=1:(length(geneNames))                       # for each row in the overall patient's data matrix
    global TNBCpt6AM[k,1] = mean(TNBCpt6[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt6AM,6,"ArithmeticMean")            # save the data

<i><u> Geometric Mean </u></i>

Dropping any cells with 0's in the calculation; also keeping number of cells with non-zero expression

In [112]:
## initialize the patient specific pseudo-bulk RNA
global TNBCpt1GM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt2GM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt3GM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt4GM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt5GM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt6GM = Array{Float64}(undef,size(raw)[1],2);

In [175]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt1[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt1GM[k,1] = geomean(tmp)    #    calculate the geometric mean over all single cell RNA columns
    global TNBCpt1GM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt1GM[:,1],1,"GeometicMean")            # save the data

In [176]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt2[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt2GM[k,1] = geomean(tmp)    #    calculate the geometric mean over all single cell RNA columns
    global TNBCpt2GM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt2GM[:,1],2,"GeometicMean")            # save the data

In [177]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt3[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt3GM[k,1] = geomean(tmp)    #    calculate the geometric mean over all single cell RNA columns
    global TNBCpt3GM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt3GM[:,1],3,"GeometicMean")            # save the data

In [178]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt4[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt4GM[k,1] = geomean(tmp)    #    calculate the geometric mean over all single cell RNA columns
    global TNBCpt4GM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt4GM[:,1],4,"GeometicMean")            # save the data 

In [179]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt5[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt5GM[k,1] = geomean(tmp)    #    calculate the geometric mean over all single cell RNA columns
    global TNBCpt5GM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt5GM[:,1],5,"GeometicMean")            # save the data 

In [180]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt6[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt6GM[k,1] = geomean(tmp)    #    calculate the geometric mean over all single cell RNA columns
    global TNBCpt6GM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt6GM[:,1],6,"GeometicMean")            # save the data

<i><u> Harmonic Mean </u></i>

In [119]:
## initialize the patient specific pseudo-bulk RNA
global TNBCpt1HM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt2HM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt3HM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt4HM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt5HM = Array{Float64}(undef,size(raw)[1],2);
global TNBCpt6HM = Array{Float64}(undef,size(raw)[1],2);

In [181]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt1[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt1HM[k,1] = harmmean(tmp)   #    calculate the harmonic mean over all single cell RNA columns
    global TNBCpt1HM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt1HM[:,1],1,"HarmonicMean")             # save the data

In [182]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt2[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt2HM[k,1] = harmmean(tmp)   #    calculate the harmonic mean over all single cell RNA columns
    global TNBCpt2HM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt2HM[:,1],2,"HarmonicMean")             # save the data

In [183]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt3[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt3HM[k,1] = harmmean(tmp)   #    calculate the harmonic mean over all single cell RNA columns
    global TNBCpt3HM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt3HM[:,1],3,"HarmonicMean")             # save the data

In [184]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt4[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt4HM[k,1] = harmmean(tmp)   #    calculate the harmonic mean over all single cell RNA columns
    global TNBCpt4HM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt4HM[:,1],4,"HarmonicMean")             # save the data

In [185]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt5[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt5HM[k,1] = harmmean(tmp)   #    calculate the harmonic mean over all single cell RNA columns
    global TNBCpt5HM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt5HM[:,1],5,"HarmonicMean")             # save the data

In [186]:
for k=1:(length(geneNames))                 # for each row in the overall patient's data matrix
    global tmp = TNBCpt6[k,:];              #    creates temporary column of gene's exp across cells
    global tmp = tmp[tmp.!=0];              #    select for non zero 0 express cells only (A[A.!=0])
    global TNBCpt6HM[k,1] = harmmean(tmp)   #    calculate the harmonic mean over all single cell RNA columns
    global TNBCpt6HM[k,2] = Float64(size(tmp)[1])     # have extra column with number of non-zero exp cells
end
saveData(TNBCpt6HM[:,1],6,"HarmonicMean")             # save the data

<i><u> Generalized/Power Mean </u></i>

In [126]:
## initialize the patient specific pseudo-bulk RNA
global TNBCpt1PM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt2PM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt3PM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt4PM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt5PM = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt6PM = Array{Float64}(undef,size(raw)[1],1);

In [168]:
for k=1:(length(geneNames))                           # for each row in the overall patient's data matrix
    global TNBCpt1PM[k,1] = genmean(TNBCpt1[k,:],2)   #   calculate the gen/power mean over all single cell RNA columns
end
saveData(TNBCpt1PM,1,"PowerMean")                      # save the data

In [167]:
for k=1:(length(geneNames))                           # for each row in the overall patient's data matrix
    global TNBCpt2PM[k,1] = genmean(TNBCpt2[k,:],2)   #   calculate the gen/power mean over all single cell RNA columns
end
saveData(TNBCpt2PM,2,"PowerMean")                      # save the data

In [166]:
for k=1:(length(geneNames))                           # for each row in the overall patient's data matrix
    global TNBCpt3PM[k,1] = genmean(TNBCpt3[k,:],2)   #   calculate the gen/power mean over all single cell RNA columns
end
saveData(TNBCpt3PM,3,"PowerMean")                      # save the data 

In [165]:
for k=1:(length(geneNames))                           # for each row in the overall patient's data matrix
    global TNBCpt4PM[k,1] = genmean(TNBCpt4[k,:],2)   #   calculate the gen/power mean over all single cell RNA columns
end
saveData(TNBCpt4PM,4,"PowerMean")                      # save the data

In [164]:
for k=1:(length(geneNames))                           # for each row in the overall patient's data matrix
    global TNBCpt5PM[k,1] = genmean(TNBCpt5[k,:],2)   #   calculate the gen/power mean over all single cell RNA columns
end
saveData(TNBCpt5PM,5,"PowerMean")                      # save the data

In [163]:
for k=1:(length(geneNames))                           # for each row in the overall patient's data matrix
    global TNBCpt6PM[k,1] = genmean(TNBCpt6[k,:],2)   #   calculate the gen/power mean over all single cell RNA columns
end
saveData(TNBCpt6PM,6,"PowerMean")                      # save the data

<i><u> Count Summation </i></u>

In [133]:
## initialize the patient specific pseudo-bulk RNA
global TNBCpt1S = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt2S = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt3S = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt4S = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt5S = Array{Float64}(undef,size(raw)[1],1);
global TNBCpt6S = Array{Float64}(undef,size(raw)[1],1);

In [157]:
for k=1:(length(geneNames))                     # for each row in the overall patient's data matrix
    global TNBCpt1S[k,1] = sum(TNBCpt1[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt1S,1,"Sum")                      # save the data

In [158]:
for k=1:(length(geneNames))                     # for each row in the overall patient's data matrix
    global TNBCpt2S[k,1] = sum(TNBCpt2[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt2S,2,"Sum")                      # save the data

In [159]:
for k=1:(length(geneNames))                     # for each row in the overall patient's data matrix
    global TNBCpt3S[k,1] = sum(TNBCpt3[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt3S,3,"Sum")                      # save the data

In [160]:
for k=1:(length(geneNames))                     # for each row in the overall patient's data matrix
    global TNBCpt4S[k,1] = sum(TNBCpt4[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt4S,4,"Sum")                      # save the data

In [161]:
for k=1:(length(geneNames))                     # for each row in the overall patient's data matrix
    global TNBCpt5S[k,1] = sum(TNBCpt5[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt5S,5,"Sum")                      # save the data

In [162]:
for k=1:(length(geneNames))                     # for each row in the overall patient's data matrix
    global TNBCpt6S[k,1] = sum(TNBCpt6[k,:])    #    calculate the arithmetic mean over all single cell RNA columns
end
saveData(TNBCpt6S,6,"Sum")                      # save the data